For now, starter code that compares lines from the two files and prints out the differences.

In [1]:
import json, csv
import pandas as pd
from constants import *
import numpy as np
from scipy.stats import poisson

def poisson_over_likelyhood(mean, target):
    return 1 - poisson.cdf(target, mean)



C:\Users\Aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


The following code will web scrape to generate the player data, feel free to ingore

In [2]:
import fbref_scrape
# import pp_scrape

200.  We're in
200.  We're in


KeyError: 'Standard Stats'

In [ ]:
players = pd.read_csv("Premier_League_Player_Stats.csv")
PP_lines = pd.read_csv("pp_lines.csv")

merged = pd.merge(players, PP_lines, on="name")
players

,name,games,minutes,gk_saves,clearances,tackles,assisted_shots,fouled,shots,crosses,shots_on_target,passes,interceptions,fouls,goals,assists,cards_yellow
0,William Saliba,28,2520,NaN,57.0,28.0,5.0,10.0,8.0,NaN,NaN,2235.0,21.0,14.0,2,1,3
1,Declan Rice,28,2391,NaN,43.0,55.0,29.0,10.0,40.0,NaN,NaN,1870.0,35.0,21.0,6,5,3
2,Bukayo Saka,27,2262,NaN,14.0,49.0,68.0,50.0,77.0,NaN,NaN,1220.0,10.0,30.0,13,8,3
3,Martin Odegaard,25,2208,NaN,5.0,34.0,73.0,22.0,53.0,NaN,NaN,1499.0,7.0,22.0,6,6,2
4,Ben White,27,2134,NaN,48.0,31.0,22.0,9.0,8.0,NaN,NaN,1648.0,24.0,12.0,2,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,Sam Curtis,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
709,Adam Davies,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
710,Louie Marsh,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
711,Dovydas Sasnauskas,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


In [ ]:
merged = merged.loc[merged['minutes'] > 200].loc[merged['games'] > 5]

merged.loc[merged['Goalie Saves'] > 0].head()

,name,games,minutes,gk_saves,clearances,tackles,assisted_shots,fouled,shots,crosses,...,Shots,Goal + Assist,Passes Attempted,Clearances,Fouls Drawn,Shots Assisted,Crosses,Interceptions,Fouls,Goals
10,Caoimhin Kelleher,7,630,20.0,3.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,Emiliano Martinez,27,2430,75.0,10.0,0.0,2.0,4.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,Guglielmo Vicario,27,2430,77.0,24.0,0.0,0.0,6.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,Andre Onana,28,2520,103.0,9.0,1.0,3.0,3.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Alphonse Areola,25,2204,104.0,11.0,1.0,0.0,1.0,0.0,NaN,...,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN


We need a new dataframe such that each row repesents a singular betting line that has a per-90 line, a per-game line, and a prizepicks line

In [ ]:
results = []

for index, row in merged.iterrows():
    # Access the values of each column in the current row
    name = row['name']

    for pp_stat_type in pp_to_fbref_stats.keys():
        # failsafe 
        if pp_stat_type not in row:
            continue

        if row[pp_stat_type] <= 0:
            continue

        # Create a dictionary to store the result
        result_dict = {
            'name': name,
            'stat': pp_stat_type,
            'line': row[pp_stat_type]
        }

        # Get the FBref stat type that corresponds to the current PP stat type
        fbref_stat_type = pp_to_fbref_stats[pp_stat_type]

        # Get the value of the FBref stat type for the current player
        fbref_stat_value = row[fbref_stat_type]

        # Add the FBref stat value to the result dictionary
        result_dict['per90'] = fbref_stat_value * 90 / row['minutes']
        result_dict['perGame'] = fbref_stat_value / row['games']


        # Append the dictionary to the results list
        results.append(result_dict)

# Convert the results list into a dataframe
df = pd.DataFrame(results).dropna()

df


,name,stat,line,per90,perGame
8,Virgil van Dijk,Passes Attempted,77.5,81.067194,78.884615
10,Virgil van Dijk,Fouls,0.5,0.632411,0.615385
17,Luis Diaz,Shots,2.5,3.284314,2.481481
22,Luis Diaz,Fouls,1.5,1.666667,1.259259
26,Alexis Mac Allister,Tackles,2.0,3.184080,2.782609
...,...,...,...,...,...
1321,Dara O'Shea,Clearances,4.5,5.170940,5.041667
1336,Charlie Taylor,Fouls Drawn,0.5,0.731283,0.700000
1346,Vitinho,Tackles,2.0,2.581047,2.090909
1364,Josh Cullen,Passes Attempted,57.5,58.638699,50.733333


Now that we have that sorted, we can analyze the chart for interesting discrepencies in data

In [ ]:
df = df.assign(quo = df['line'] / df['per90']).assign(diff = df['line'] - df['per90']).assign(over_likelyhood = poisson_over_likelyhood(df['per90'], df['line']))

df = df.sort_values(by='over_likelyhood', ascending=False)

# show the full thing
pd.set_option('display.max_rows', None)
df

,name,stat,line,per90,perGame,quo,diff,over_likelyhood
752,Dan Burn,Passes Attempted,31.5,57.667032,50.869565,0.546239,-26.167032,0.999911
980,Calvin Bassey,Passes Attempted,46.5,65.687812,53.894737,0.707894,-19.187812,0.993390
500,Diogo Dalot,Passes Attempted,35.5,52.440633,50.961538,0.676956,-16.940633,0.993083
488,Bruno Fernandes,Passes Attempted,43.5,61.185185,61.185185,0.710956,-17.685185,0.990936
740,Bruno Guimaraes,Passes Attempted,45.5,63.388325,61.666667,0.717798,-17.888325,0.990490
944,Joao Palhinha,Passes Attempted,32.5,43.459893,39.260870,0.747816,-10.959893,0.956769
200,Kyle Walker,Passes Attempted,68.5,79.765487,77.038462,0.858767,-11.265487,0.898632
416,Yves Bissouma,Passes Attempted,72.5,82.237197,71.368421,0.881596,-9.737197,0.859317
296,Moussa Diaby,Passes Attempted,21.5,26.598952,16.107143,0.808303,-5.098952,0.838778
653,Danny Ings,Shots,1.5,3.068182,0.642857,0.488889,-1.568182,0.810807
